In [106]:
spark.stop()

In [ ]:
import init

import findspark
findspark.init()

import os
import pandas as pd
import numpy as np
from pyspark.sql import *
from pyspark import *
import pyspark.sql.functions as f
from pyspark.sql.types import *
import time
from pyrecdp.data_processor import *
from pyrecdp.encoder import *
from pyrecdp.utils import *

spark = SparkSession.builder.master('yarn')\
        .appName("Recsys2021_data_process")\
        .getOrCreate()
path_prefix = "hdfs://"
dicts_folder = "recsys_dicts/"

In [4]:
train_full = spark.read.parquet("/recsys2021_0608")
#train_full.printSchema()
train_full.count()
# df = train_full.drop('hashtags')\
#                .withColumnRenamed('original_hashtags', 'hashtags')
# df = df.select('hashtags').withColumn('hashtags', f.explode(f.split(f.col('hashtags'), '\t')))
# df = df.groupby('hashtags').agg(f.count(f.lit(1)).alias('count'))
# df.write.format('parquet').mode('overwrite').save('file:///mnt/nvme2/chendi/BlueWhale/sample_0_3/stage4_20days/lookup_table_hashtags')

626242930

In [14]:
user_hashtags_dicts = spark.read.parquet('file:///mnt/nvme2/chendi/BlueWhale/sample_0_3/stage4_20days/add_chendi_feature/rsc_0622_best/lookup_table_engaged_with_user_id_response')
lookup_table = spark.read.parquet("/recsys2021_0608_processed/sample_0_3_20days_eric_features/valid_lookup/lookup_table_engaged_with_user_id_fold")
df = lookup_table.join(user_hashtags_dicts, 'engaged_with_user_id', 'left')
df.write.format('parquet').mode('overwrite').save('file:///mnt/nvme2/chendi/BlueWhale/sample_0_3/stage4_20days/lookup_table_engaged_with_user_id_fold')

In [16]:
df = spark.read.parquet("file:///mnt/nvme2/chendi/BlueWhale/sample_0_3/stage4_20days/lookup_table_engaged_with_user_id_fold")
df.show(vertical=True)

-RECORD 0--------------------------------------------------------------------------------
 engaged_with_user_id                                             | 00001556ED74C7C92... 
 TE_engaged_with_user_id_reply_timestamp_indicator                | 0.027006638950719367 
 TE_engaged_with_user_id_retweet_timestamp_indicator              | 0.08696896238375852  
 TE_engaged_with_user_id_retweet_with_comment_timestamp_indicator | 0.006716167120638... 
 TE_engaged_with_user_id_like_timestamp_indicator                 | 0.39640563540158563  
 avg_first_engage_elapse                                          | 136476.5             
 avg_first_reply_elapse                                           | 259200.0             
 avg_first_retweet_elapse                                         | 259200.0             
 avg_first_comment_elapse                                         | 259200.0             
 avg_first_like_elapse                                            | 136476.5             
-RECORD 1-

In [42]:
import pyspark.sql.functions as F
train_full = spark.read.parquet("/recsys2021_0608_processed/tweet_text_processed_fixed")
df = train_full.withColumn('mention', F.regexp_extract(F.col('original_tweet'), r"[^RT]\s@(\S+)", 1)).select('engaging_user_id', 'mention')
df = df.groupby('engaging_user_id').agg(f.collect_set('mention').alias('mention_intersection_prescence_unique'))
df.write.format('parquet').mode('overwrite').save('file:///mnt/nvme2/chendi/BlueWhale/sample_0_3/stage4_20days/CPD_mention_engaging_user_id')

In [45]:
valid = spark.read.parquet("file:///mnt/nvme2/chendi/BlueWhale/sample_0_3/stage4_20days/add_chendi_feature/rsc_0622_best/validate_with_features")
train = spark.read.parquet("file:///mnt/nvme2/chendi/BlueWhale/sample_0_3/stage4_20days/add_chendi_feature/rsc_0622_best/train_with_features_fold_0.03")

In [58]:
lookup_eric_list = [
    "lookup_table_hashtags",
    #"CPD_mention_engaging_user_id"
]
eric_dfs = [(fn, spark.read.parquet("file:///mnt/nvme2/chendi/BlueWhale/sample_0_3/stage4_20days/%s" % (fn))) for fn in lookup_eric_list]
fe_list = {}
for fn, dict_df in eric_dfs:
    fe_list[fn] = dict_df.columns

In [9]:
# def fudf(val):
#     o = []
#     for l in val:
#         o += l
#     return o
# 
# flattenUdf = f.udf(fudf, ArrayType(StringType()))
# 
# after_agg_fold = []
# for fn, dict_df in eric_dfs:
#     to_select = list(set(fe_list[fn]) - set(['fold', 'engaging_user_id']))
#     df = dict_df.groupby('engaging_user_id').agg(f.collect_list(to_select[0]).alias(to_select[0])).withColumn(to_select[0], flattenUdf(f"{to_select[0]}").alias(to_select[0]))
#     after_agg_fold.append((fn, df))

+--------------------+--------------------+
|min(tweet_timestamp)|max(tweet_timestamp)|
+--------------------+--------------------+
|          1612396800|          1614124799|
+--------------------+--------------------+

In [53]:
df=train
time_range = [(1612396800, 1613952000), (1613952000, 1614124799)]
#df.filter((f.col('tweet_timestamp') > f.lit(time_range[0][0])) & (f.col('tweet_timestamp') < f.lit(time_range[0][1])))\
#  .write.format('parquet').save("file:///mnt/nvme2/chendi/BlueWhale/sample_0_3/stage4_20days/add_chendi_feature/rsc_0622_best/train_with_features_fold_0.03_18days")

df.filter((f.col('tweet_timestamp') > f.lit(time_range[1][0])) & (f.col('tweet_timestamp') <= f.lit(time_range[1][1])))\
  .write.format('parquet').save("file:///mnt/nvme2/chendi/BlueWhale/sample_0_3/stage4_20days/add_chendi_feature/rsc_0622_best/train_with_features_fold_0.03_2days")

In [56]:
df = spark.read.parquet("file:///mnt/nvme2/chendi/BlueWhale/sample_0_3/stage4_20days/lookup_table_hashtags")
df.count()

6322547

In [36]:
df = train
to_select = []
for fn, dict_df in eric_dfs:
    df = df.join(dict_df.hint('shuffle_hash'), ['hashtags'], 'left')
    to_select += list(set(fe_list[fn]) - set(['fold', 'engaging_user_id']))
for l in to_select:
    if 'hashtags' in l:
        df = df.withColumn('hashtags', f.when(f.col('hashtags') == '', None).otherwise(f.split(f.col('hashtags'), '\t')))
        df = df.withColumn('hashtags_intersection_cnt', f.when(f.size(f.col('hashtags'))>0, f.size(f.array_intersect(f.col('hashtags'), f.col(l)))).otherwise(0))
        df.write.format('parquet').mode('overwrite').save('file:///mnt/nvme2/chendi/BlueWhale/sample_0_3/stage4_20days/add_chendi_feature/rsc_0622_best/train_with_features_fold_0.03_withCPD')
        #df.filter(f'{l} is not null').select('engaging_user_id', 'hashtags', l, 'hashtags_intersection_cnt').show(vertical=True, truncate=50)

In [62]:
df = train
fn, dict_df = eric_dfs[0]
df = df.withColumn('row_id', f.monotonically_increasing_id())
df.write.format('parquet').mode('overwrite').save('tmp_with_row_id')
df = spark.read.parquet('tmp_with_row_id')
df = df.withColumn('hashtags', f.when(f.col('hashtags') == '', None).otherwise(f.split(f.col('hashtags'), '\t')))
new_df = df.select('row_id', 'hashtags').withColumn('hashtags', f.explode(f.col('hashtags')))
new_df = new_df.join(dict_df, 'hashtags', 'left').withColumnRenamed('count', 'hashtags_cnt')
new_df = new_df.groupby('row_id').agg(f.collect_list(f.col('hashtags_cnt')).alias('hashtags_cnt'))
df = df.join(new_df, 'row_id', 'left').drop('row_id')
df = df.withColumn("top_hashtags_cnt", f.array_max(f.col('hashtags_cnt')))
df.write.format('parquet').mode('overwrite').save('file:///mnt/nvme2/chendi/BlueWhale/sample_0_3/stage4_20days/add_chendi_feature/rsc_0622_best/train_with_features_0.03_withHashtagsCnt')
#df.filter(f'{l} is not null').select('engaging_user_id', 'hashtags', l, 'hashtags_intersection_cnt').show(vertical=True, truncate=50)

In [ ]:
df = spark.read.parquet("/recsys2021_0608").select()
qd = QuantileDiscretizer(numBuckets=100, inputCol="engaged_with_user_follower_count", outputCol='popular_bucket_id')
result  = qd.fit(df).transform(df)
#result.groupby('bucket_id').agg(F.min('count'), F.count(F.lit(1))).show()
result.write.format('parquet').mode('overwrite').save('/recsys2021_0608_processed/recsys_dicts/tweet_word_with_bucketid')

In [5]:
#train = spark.read.parquet('file:///mnt/nvme2/chendi/BlueWhale/sample_0_3/stage4_20days/add_chendi_feature/rsc_0622_best/train_with_features_0.03_withHashtagsCnt')
#original_data = spark.read.parquet("/recsys2021_0608").select('tweet_id', 'engaging_user_id', 'reply_timestamp', 'retweet_timestamp', 'retweet_with_comment_timestamp', 'like_timestamp')
#train = train.drop('reply_timestamp').drop('retweet_timestamp').drop('retweet_with_comment_timestamp').drop('like_timestamp').join(original_data, ['tweet_id', 'engaging_user_id'], 'left')
#train.write.format('parquet').mode('overwrite').save('file:///mnt/nvme2/chendi/BlueWhale/sample_0_3/stage4_20days/add_chendi_feature/rsc_0622_best/train_with_features_0.03_withHashtagsCnt_ts')
#train = spark.read.parquet('file:///mnt/nvme2/chendi/BlueWhale/sample_0_3/stage4_20days/add_chendi_feature/rsc_0622_best/train_with_features_0.03_withHashtagsCnt_ts')
train = spark.read.parquet("/recsys2021_0608").select('tweet_id', 'engaged_with_user_id', 'tweet_timestamp', 'reply_timestamp', 'retweet_timestamp', 'retweet_with_comment_timestamp', 'like_timestamp')
def py_get_follower_bucket_id(follower):
    return (follower_quantile.index(min(follower_quantile, key=lambda x:abs(x-follower))) + 1)
get_follower_bucket_id = f.udf(lambda x: py_get_follower_bucket_id(x))

#train = train.withColumn('follower_bucket_id', get_follower_bucket_id('engaged_with_user_follower_count'))

df = train.withColumn('first_reply_elapse', f.col('reply_timestamp') - f.col('tweet_timestamp'))\
     .withColumn('first_retweet_elapse', f.col('retweet_timestamp') - f.col('tweet_timestamp'))\
     .withColumn('first_comment_elapse', f.col('retweet_with_comment_timestamp') - f.col('tweet_timestamp'))\
     .withColumn('first_like_elapse', f.col('like_timestamp') - f.col('tweet_timestamp'))\
     .fillna(259200, ['first_reply_elapse', 'first_retweet_elapse', 'first_comment_elapse', 'first_like_elapse'])\
     .groupby('tweet_id')\
     .agg(
        f.first('engaged_with_user_id').alias('engaged_with_user_id'),
        f.min('first_reply_elapse').alias('first_reply_elapse'),
        f.min('first_retweet_elapse').alias('first_retweet_elapse'),
        f.min('first_comment_elapse').alias('first_comment_elapse'),
        f.min('first_like_elapse').alias('first_like_elapse'))\
     .withColumn('first_engage_elapse', f.least(f.col('first_reply_elapse'), f.col('first_retweet_elapse'), f.col('first_comment_elapse'), f.col('first_like_elapse')))\
     .groupby('engaged_with_user_id')\
     .agg(
        f.mean('first_engage_elapse').alias('avg_first_engage_elapse'),
        f.mean('first_reply_elapse').alias('avg_first_reply_elapse'),
        f.mean('first_retweet_elapse').alias('avg_first_retweet_elapse'),
        f.mean('first_comment_elapse').alias('avg_first_comment_elapse'),
        f.mean('first_like_elapse').alias('avg_first_like_elapse')
     )
#df.write.format('parquet').mode('overwrite').save('file:///mnt/nvme2/chendi/BlueWhale/sample_0_3/stage4_20days/add_chendi_feature/rsc_0622_best/lookup_table_follower_bucket_id_response')
df.write.format('parquet').mode('overwrite').save('file:///mnt/nvme2/chendi/BlueWhale/sample_0_3/stage4_20days/add_chendi_feature/rsc_0622_best/lookup_table_engaged_with_user_id_response')
#df.select('engaged_with_user_id', 'avg_first_engage_elapse', 'avg_first_reply_elapse', 'avg_first_retweet_elapse', 'avg_first_comment_elapse', 'avg_first_like_elapse').show(vertical=True)


In [8]:
response_by_user_df = spark.read.parquet('file:///mnt/nvme2/chendi/BlueWhale/sample_0_3/stage4_20days/add_chendi_feature/rsc_0622_best/lookup_table_engaged_with_user_id_response')
response_by_follower_bucket_id_df = spark.read.parquet('file:///mnt/nvme2/chendi/BlueWhale/sample_0_3/stage4_20days/add_chendi_feature/rsc_0622_best/lookup_table_follower_bucket_id_response')
response_by_follower_bucket_id_df = response_by_follower_bucket_id_df\
    .withColumnRenamed('avg_first_engage_elapse', 'avg_first_engage_elapse_by_group')\
    .withColumnRenamed('avg_first_reply_elapse', 'avg_first_reply_elapse_by_group')\
    .withColumnRenamed('avg_first_retweet_elapse', 'avg_first_retweet_elapse_by_group')\
    .withColumnRenamed('avg_first_comment_elapse', 'avg_first_comment_elapse_by_group')\
    .withColumnRenamed('avg_first_like_elapse', 'avg_first_like_elapse_by_group')

train = spark.read.parquet('file:///mnt/nvme2/chendi/BlueWhale/sample_0_3/stage4_20days/add_chendi_feature/rsc_0622_best/train_with_features_fold_0.03')
train = train.withColumn('follower_bucket_id', get_follower_bucket_id('engaged_with_user_follower_count'))
df = train.join(response_by_user_df, 'engaged_with_user_id', 'left')
df = df.join(response_by_follower_bucket_id_df, 'follower_bucket_id', 'left')
df.write.format('parquet').mode('overwrite').save('file:///mnt/nvme2/chendi/BlueWhale/sample_0_3/stage4_20days/add_chendi_feature/rsc_0622_best/train_with_features_0.03_response')


In [10]:
response_by_user_df = spark.read.parquet('file:///mnt/nvme2/chendi/BlueWhale/sample_0_3/stage4_20days/add_chendi_feature/rsc_0622_best/lookup_table_engaged_with_user_id_response')
response_by_follower_bucket_id_df = spark.read.parquet('file:///mnt/nvme2/chendi/BlueWhale/sample_0_3/stage4_20days/add_chendi_feature/rsc_0622_best/lookup_table_follower_bucket_id_response')
response_by_follower_bucket_id_df = response_by_follower_bucket_id_df\
    .withColumnRenamed('avg_first_engage_elapse', 'avg_first_engage_elapse_by_group')\
    .withColumnRenamed('avg_first_reply_elapse', 'avg_first_reply_elapse_by_group')\
    .withColumnRenamed('avg_first_retweet_elapse', 'avg_first_retweet_elapse_by_group')\
    .withColumnRenamed('avg_first_comment_elapse', 'avg_first_comment_elapse_by_group')\
    .withColumnRenamed('avg_first_like_elapse', 'avg_first_like_elapse_by_group')

valid = spark.read.parquet("file:///mnt/nvme2/chendi/BlueWhale/sample_0_3/stage4_20days/add_chendi_feature/rsc_0622_best/validate_with_features")
valid = valid.withColumn('follower_bucket_id', get_follower_bucket_id('engaged_with_user_follower_count'))

df = valid.join(response_by_user_df, 'engaged_with_user_id', 'left')
df = df.join(response_by_follower_bucket_id_df, 'follower_bucket_id', 'left')
df.write.format('parquet').mode('overwrite').save('file:///mnt/nvme2/chendi/BlueWhale/sample_0_3/stage4_20days/add_chendi_feature/rsc_0622_best/validate_with_features_response')


In [11]:
df = spark.read.parquet("file:///mnt/nvme2/chendi/BlueWhale/sample_0_3/stage4_20days/add_chendi_feature/rsc_0622_best/train_with_features_0.03_response")
df.select(
    "engaged_with_user_id",
    "engaged_with_user_follower_count",
    "follower_bucket_id",
    "avg_first_engage_elapse",
    "avg_first_reply_elapse",
    "avg_first_retweet_elapse",
    "avg_first_comment_elapse",
    "avg_first_like_elapse",
    "avg_first_engage_elapse_by_group",
    "avg_first_reply_elapse_by_group",
    "avg_first_retweet_elapse_by_group",
    "avg_first_comment_elapse_by_group",
    "avg_first_like_elapse_by_group"
).show(vertical=True)

-RECORD 0-------------------------------------------------
 engaged_with_user_id              | 0000F3881E43889D4... 
 engaged_with_user_follower_count  | 2448                 
 follower_bucket_id                | 90                   
 avg_first_engage_elapse           | 97529.79166666667    
 avg_first_reply_elapse            | 259200.0             
 avg_first_retweet_elapse          | 205273.625           
 avg_first_comment_elapse          | 248404.39583333334   
 avg_first_like_elapse             | 151454.77083333334   
 avg_first_engage_elapse_by_group  | 137044.72750202756   
 avg_first_reply_elapse_by_group   | 248878.21071213257   
 avg_first_retweet_elapse_by_group | 230296.9214059211    
 avg_first_comment_elapse_by_group | 256631.77396761082   
 avg_first_like_elapse_by_group    | 161268.630979666     
-RECORD 1-------------------------------------------------
 engaged_with_user_id              | 0000F3881E43889D4... 
 engaged_with_user_follower_count  | 2448               

In [2]:
follower_quantile = [
 9.0,
 15.0,
 20.0,
 24.0,
 28.0,
 33.0,
 37.0,
 41.0,
 46.0,
 50.0,
 54.0,
 59.0,
 63.0,
 68.0,
 73.0,
 77.0,
 82.0,
 87.0,
 92.0,
 97.0,
 102.0,
 108.0,
 113.0,
 119.0,
 124.0,
 130.0,
 136.0,
 142.0,
 148.0,
 154.0,
 160.0,
 167.0,
 174.0,
 181.0,
 188.0,
 195.0,
 203.0,
 211.0,
 218.0,
 227.0,
 235.0,
 244.0,
 253.0,
 262.0,
 271.0,
 281.0,
 291.0,
 302.0,
 313.0,
 324.0,
 335.0,
 347.0,
 359.0,
 372.0,
 386.0,
 400.0,
 415.0,
 430.0,
 445.0,
 462.0,
 479.0,
 497.0,
 517.0,
 536.0,
 557.0,
 580.0,
 603.0,
 628.0,
 654.0,
 683.0,
 713.0,
 745.0,
 780.0,
 818.0,
 858.0,
 902.0,
 950.0,
 1003.0,
 1056.0,
 1114.0,
 1178.0,
 1249.0,
 1329.0,
 1420.0,
 1525.0,
 1646.0,
 1788.0,
 1957.0,
 2150.0,
 2383.0,
 2674.0,
 3047.0,
 3521.0,
 4168.0,
 5110.0,
 6559.0,
 9096.0,
 14008.0,
 29093.0]

In [103]:
train = spark.read.parquet('file:///mnt/nvme2/chendi/BlueWhale/sample_0_3/stage4_20days/add_chendi_feature/rsc_0622_best/train_with_features_0.03_response')
valid = spark.read.parquet('file:///mnt/nvme2/chendi/BlueWhale/sample_0_3/stage4_20days/add_chendi_feature/rsc_0622_best/validate_with_features_response')

def py_get_follower_bucket_id(follower):
    return (follower_quantile.index(min(follower_quantile, key=lambda x:abs(x-follower))) + 1)
get_follower_bucket_id = f.udf(lambda x: py_get_follower_bucket_id(x))

train = train.withColumn('follower_bucket_id', get_follower_bucket_id('engaged_with_user_follower_count'))
train.write.format('parquet').mode('overwrite').save('file:///mnt/nvme2/chendi/BlueWhale/sample_0_3/stage4_20days/add_chendi_feature/rsc_0622_best/train_with_features_0.03_follower_bucket_response')

valid = valid.withColumn('follower_bucket_id', get_follower_bucket_id('engaged_with_user_follower_count'))
valid.write.format('parquet').mode('overwrite').save('file:///mnt/nvme2/chendi/BlueWhale/sample_0_3/stage4_20days/add_chendi_feature/rsc_0622_best/validate_with_features_follower_bucket_response')
#train.select('tweet_id', 'follower_bucket_id', 'engaged_with_user_follower_count', 'engaged_with_user_id', 'avg_first_engage_elapse', 'avg_first_reply_elapse', 'avg_first_retweet_elapse', 'avg_first_comment_elapse', 'avg_first_like_elapse').show(vertical=True)
# df = train.groupby('follower_bucket_id')\
#           .agg(f.mean('avg_first_engage_elapse').alias('avg_first_engage_elapse_by_follower_group'),
#                f.mean('avg_first_reply_elapse').alias('avg_first_reply_elapse_by_follower_group'),
#                f.mean('avg_first_retweet_elapse').alias('avg_first_retweet_elapse_by_follower_group'),
#                f.mean('avg_first_comment_elapse').alias('avg_first_comment_elapse_by_follower_group'),
#                f.mean('avg_first_like_elapse').alias('avg_first_like_elapse_by_follower_group'))
# df.write.format('parquet').mode('overwrite').save('file:///mnt/nvme2/chendi/BlueWhale/sample_0_3/stage4_20days/add_chendi_feature/rsc_0622_best/user_follower_bucket_id_response')

In [63]:
df = valid
fn, dict_df = eric_dfs[0]
df = df.withColumn('row_id', f.monotonically_increasing_id())
df.write.format('parquet').mode('overwrite').save('tmp_with_row_id')
df = spark.read.parquet('tmp_with_row_id')
df = df.withColumn('hashtags', f.when(f.col('hashtags') == '', None).otherwise(f.split(f.col('hashtags'), '\t')))
new_df = df.select('row_id', 'hashtags').withColumn('hashtags', f.explode(f.col('hashtags')))
new_df = new_df.join(dict_df, 'hashtags', 'left').withColumnRenamed('count', 'hashtags_cnt')
new_df = new_df.groupby('row_id').agg(f.collect_list(f.col('hashtags_cnt')).alias('hashtags_cnt'))
df = df.join(new_df, 'row_id', 'left').drop('row_id')
df = df.withColumn("top_hashtags_cnt", f.array_max(f.col('hashtags_cnt')))
df.write.format('parquet').mode('overwrite').save('file:///mnt/nvme2/chendi/BlueWhale/sample_0_3/stage4_20days/add_chendi_feature/rsc_0622_best/validate_with_features_0.03_withHashtagsCnt')
#df.filter(f'{l} is not null').select('engaging_user_id', 'hashtags', l, 'hashtags_intersection_cnt').show(vertical=True, truncate=50)

In [66]:
new_train = spark.read.parquet('file:///mnt/nvme2/chendi/BlueWhale/sample_0_3/stage4_20days/add_chendi_feature/rsc_0622_best/train_with_features_0.03_withHashtagsCnt')
new_train.select("hashtags", 'hashtags_cnt', 'top_hashtags_cnt').show(vertical=True, truncate=100)

-RECORD 0----------------------------------------------------------------------------------------------------------------
 hashtags         | null                                                                                                 
 hashtags_cnt     | null                                                                                                 
 top_hashtags_cnt | null                                                                                                 
-RECORD 1----------------------------------------------------------------------------------------------------------------
 hashtags         | [E27B360437006B7CDB55A879997511CF]                                                                   
 hashtags_cnt     | [137]                                                                                                
 top_hashtags_cnt | 137                                                                                                  
-RECORD 2---------------

In [37]:
df = valid
to_select = []
for fn, dict_df in eric_dfs:
    df = df.join(dict_df.hint('shuffle_hash'), ['engaging_user_id'], 'left')
    to_select += list(set(fe_list[fn]) - set(['fold', 'engaging_user_id']))
# df.write.format('parquet').mode('overwrite').save('file:///mnt/nvme2/chendi/BlueWhale/sample_0_3/stage4_20days/add_chendi_feature/rsc_0622_best/train_with_features_fold_0.03_withCPD')
# df = spark.read.parquet('file:///mnt/nvme2/chendi/BlueWhale/sample_0_3/stage4_20days/add_chendi_feature/rsc_0622_best/train_with_features_fold_0.03_withCPD')
# df.select('engaging_user_id', 'hashtags_intersection_prescence_unique').show()
for l in to_select:
    if 'hashtags' in l:
        df = df.withColumn('hashtags', f.when(f.col('hashtags') == '', None).otherwise(f.split(f.col('hashtags'), '\t')))
        df = df.withColumn('hashtags_intersection_cnt', f.when(f.size(f.col('hashtags'))>0, f.size(f.array_intersect(f.col('hashtags'), f.col(l)))).otherwise(0))
        df.write.format('parquet').mode('overwrite').save('file:///mnt/nvme2/chendi/BlueWhale/sample_0_3/stage4_20days/add_chendi_feature/rsc_0622_best/validate_with_features_fold_0.03_withCPD')
        #df.filter(f'{l} is not null').select('engaging_user_id', 'hashtags', l, 'hashtags_intersection_cnt').show(vertical=True, truncate=50)

In [40]:
new_valid = spark.read.parquet('file:///mnt/nvme2/chendi/BlueWhale/sample_0_3/stage4_20days/add_chendi_feature/rsc_0622_best/validate_with_features_fold_0.03_withCPD')
new_valid.filter('hashtags is not null').select('engaging_user_id', 'hashtags', 'hashtags_intersection_cnt').show()

+--------------------+--------------------+-------------------------+
|    engaging_user_id|            hashtags|hashtags_intersection_cnt|
+--------------------+--------------------+-------------------------+
|7901BC455B9E216D8...|[B9926899C4D859E9...|                        0|
|D00CA58804DD5567C...|[0DA4E41C3E67EC14...|                        0|
|73CD3CD031745B667...|[ADA9189B0A93DA9E...|                        1|
|596C6E8C9037C5305...|[2FD2D8F4C4067CBA...|                        1|
|04902DB299C3E3D85...|[A16DAC4C876B8FF6...|                        0|
|04E65599BFF7F54ED...|[877B9CFA67489D01...|                        0|
|04E65599BFF7F54ED...|[A78E011F11A8E13E...|                        0|
|04E65599BFF7F54ED...|[318C847E7A67D037...|                        0|
|04E65599BFF7F54ED...|[8769068209E23F83...|                        0|
|16CA0CBBF272D7DD0...|[985DC0FB72A3E7A8...|                        1|
|16CA0CBBF272D7DD0...|[8B835F7982B0AF9D...|                        1|
|16CA0CBBF272D7DD0..

In [20]:
df = train
to_select = []
for fn, dict_df in eric_dfs:
    df = df.join(dict_df.hint('shuffle_hash'), ['engaging_user_id'], 'left')
    to_select += list(set(fe_list[fn]) - set(['fold', 'engaging_user_id']))
# df.write.format('parquet').mode('overwrite').save('file:///mnt/nvme2/chendi/BlueWhale/sample_0_3/stage4_20days/add_chendi_feature/rsc_0622_best/train_with_features_fold_0.03_withCPD')
# df = spark.read.parquet('file:///mnt/nvme2/chendi/BlueWhale/sample_0_3/stage4_20days/add_chendi_feature/rsc_0622_best/train_with_features_fold_0.03_withCPD')
# df.select('engaging_user_id', 'hashtags_intersection_prescence_unique').show()
for l in to_select:
    if 'hashtags' in l:
        df = df.withColumn('hashtags', f.split(f.col('hashtags'), '\t'))
        df = df.withColumn('hashtags_intersection_cnt', f.size(f.array_intersect(f.col('hashtags'), f.col(l))))
        df.write.format('parquet').mode('overwrite').save('file:///mnt/nvme2/chendi/BlueWhale/sample_0_3/stage4_20days/add_chendi_feature/rsc_0622_best/train_with_features_fold_0.03_withCPD')
        #df.filter(f'{l} is not null').select('engaging_user_id', 'hashtags', l, 'hashtags_intersection_cnt').show(vertical=True, truncate=50)